Шаг 1. Выбрать постановку задачи для семплирования и соответствующие целевые переменные из Вашего датасета.​

Шаг 2. Используя одномерные параметрические распределения, подобранные в прошлом задании, выполнить генеративное моделирование целевых переменных.​

Шаг 3. Выбрать дополнительные переменные, предположительно взаимосвязанные с целевыми, в том числе имеющие значимые коэффициенты корреляции.​

Шаг 4. Построить байесовскую сеть для выбранного набора переменных, задав в ней связи (структуру) на основе многомерного статистического анализа и обучить распределения в узлах с использованием одного из предложенных алгоритмов. ​

Шаг 5. Построить байесовскую сеть для выбранного набора переменных методами структурного обучения на более широком наборе признаков и обучить распределения в узлах с использованием одного из предложенных алгоритмов. ​

Шаг 6. Проанализировать качество семплируемой целевой переменной в контексте постановки задачи (предсказание).

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from pgmpy.factors.discrete import TabularCPD
from copy import  copy
import networkx as nx


from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel
from pgmpy.sampling import BayesianModelSampling
from pgmpy.base import DAG

In [ ]:
# Импорт данных из csv-файла - 
# ежедневные данные о погоде с многочисленных австралийских метеостанций.

path_to_file = 'weatherAUS.csv'
source_df = pd.read_csv(path_to_file, engine='python')

source_df.head(5)

Шаг 1. Выбрать постановку задачи для семплирования и соответствующие целевые переменные из Вашего датасета.​

In [ ]:
#Отбираем нужные параметры
new_df = source_df[['Temp3pm','Evaporation','Sunshine']]

#Удаление значений Nan
#Можно заполнить средними, если мало пропусков.
new_df = new_df.dropna()

new_df.head(5)

Из выбранного датасета были выбраны следующие три переменные: 


Temp3pm -- температура в три часа дня.

Evaporation -- испарения;

Sunshine -- солнечный свет.

Задача: предсказание велечины испарения.

Шаг 4. Построить байесовскую сеть для выбранного набора переменных, задав в ней связи (структуру) на основе многомерного статистического анализа и обучить распределения в узлах с использованием одного из предложенных алгоритмов.

In [ ]:
# Создание байесовской сети.
bn = BayesianModel()

# Добавление вершин.
bn.add_node("Temp3pm")
bn.add_node("Evaporation")
bn.add_node("Sunshine")

# Добавление ребер. Temp3pm	Evaporation	Sunshine
bn.add_edge("Temp3pm", "Evaporation")
bn.add_edge("Temp3pm", "Sunshine")
bn.add_edge("Evaporation", "Sunshine")

# Добавление весов.
cpd_A = TabularCPD('Temp3pm', 2, values=[[.6],[.4]])
cpd_B = TabularCPD('Evaporation', 2, values=[[.5],[.5]],
                  evidence=['Temp3pm'],
                  evidence_card=[2])
cpd_С = TabularCPD('Sunshine', 2, values=[[.4],[.6]],
                  evidence=['Temp3pm','Evaporation'],
                  evidence_card=[2,2])

bn.add_cpds(cpd_A, cpd_B, cpd_С)

In [ ]:
# # Проверка модели.
# bn.check_model()
# print("Model is correct.")

In [ ]:
def draw_comparative_hist (parametr: str, original_data: pd.DataFrame, data_sampled: pd.DataFrame):
    final_df = pd.DataFrame()
    df1 = pd.DataFrame()
    df1[parametr] = original_data[parametr]
    df1['Data'] = 'Original data'
    df1['Probability'] = df1[parametr].apply(lambda x: (df1.groupby(parametr)[parametr].count()[x])/original_data.shape[0])
    df2 = pd.DataFrame()
    df2[parametr] = data_sampled[parametr]
    df2['Data'] = 'Synthetic data'
    df2['Probability'] = df2[parametr].apply(lambda x: (df2.groupby(parametr)[parametr].count()[x])/data_sampled.shape[0])
    final_df = pd.concat([df1, df2])
    sns.barplot(x=parametr, y="Probability", hue="Data", data=final_df)
    plt.show()

In [ ]:
# Сэмплинг данных.
sampler = BayesianModelSampling(bn)
sample = sampler.forward_sample(size=, return_type='dataframe')

In [ ]:
# Построение гистограммы сравнения.
draw_comparative_hist('Evaporation',transformed_data,sample)

Шаг 5. Построить байесовскую сеть для выбранного набора переменных методами структурного обучения на более широком наборе признаков и обучить распределения в узлах с использованием одного из предложенных алгоритмов. 

In [ ]:
transformed_data = copy(new_df)
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
data_discrete = est.fit_transform(new_df.values[:,0:3])
transformed_data[['Temp3pm','Evaporation','Sunshine']] = data_discrete

In [ ]:
hc = HillClimbSearch(transformed_data, scoring_method=K2Score(transformed_data))

In [ ]:
best_model = hc.estimate()

In [ ]:
G_K2 = nx.DiGraph()
G_K2.add_edges_from(best_model.edges())
pos = nx.layout.circular_layout(G_K2)
nx.draw(G_K2, pos, with_labels=True,font_weight='bold')

In [ ]:
blacklist = [(x, y) for x in transformed_data.columns.to_list() for y in ['WindGustSpeed', 'Temp3pm'] if x != y]
best_model_new = hc.estimate(black_list=blacklist)


Шаг 6. Проанализировать качество семплируемой целевой переменной в контексте постановки задачи (предсказание).

In [ ]:
hc_BicScore = HillClimbSearch(transformed_data, scoring_method=BicScore(transformed_data))
best_model_BicScore = hc_BicScore.estimate(black_list=blacklist)

In [ ]:
G_BicScore = nx.DiGraph()
G_BicScore.add_edges_from(best_model_BicScore.edges())
pos = nx.layout.circular_layout(G_BicScore)
nx.draw(G_BicScore, pos, with_labels=True,font_weight='bold')

In [ ]:
def sampling (bn: DAG, data: pd.DataFrame, n: int = 100):
    bn_new = BayesianModel(bn.edges())
    bn_new.fit(data)
    sampler = BayesianModelSampling(bn_new)
    sample = sampler.forward_sample(size=n, return_type='dataframe')
    return sample

In [ ]:
sample_Bic = sampling(best_model_BicScore, transformed_data, 46193)

In [ ]:
sns.distplot(new_df['Evaporation'], label='Original data')
sns.distplot(sample_Bic['Evaporation'], label='Generated data')
plt.legend()

In [ ]:
sample_k2 = sampling(best_model_new, transformed_data, 46193)

In [ ]:
draw_comparative_hist('Evaporation',transformed_data,sample_Bic)

In [ ]:
draw_comparative_hist('Evaporation',transformed_data,sample_k2)